In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import  *

In [2]:
spark = SparkSession.builder.appName("Hey").getOrCreate()
json_file_path = "/Users/nombauser/Desktop/GIT/MyGitRepos/Learn-PySpark/files/sample_2.json"

25/10/06 16:30:46 WARN Utils: Your hostname, MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
25/10/06 16:30:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/10/06 16:30:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.format("json") \
    .option("multiline", "false") \
    .load(json_file_path)

In [7]:
df.printSchema()

root
 |-- amazon_price: double (nullable = true)
 |-- boardgame: string (nullable = true)
 |-- collection_stats: struct (nullable = true)
 |    |-- for_trade: long (nullable = true)
 |    |-- own: long (nullable = true)
 |    |-- previously_owned: long (nullable = true)
 |    |-- want_in_trade: long (nullable = true)
 |    |-- wishlist: long (nullable = true)
 |-- credits: struct (nullable = true)
 |    |-- artists: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- designers: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- developer: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- editor: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- graphic_designer: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- insert_designer: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 | 

In [4]:
# Selecting specific columns
df.select(
    "boardgame",
    "amazon_price",
    "credits.developer"
).show(n=4, truncate=False)

+-------------------------+------------+-----------------+
|boardgame                |amazon_price|developer        |
+-------------------------+------------+-----------------+
|Brass: Birmingham        |69.95       |null             |
|Pandemic Legacy: Season 1|76.97       |null             |
|Ark Nova                 |71.81       |null             |
|Gloomhaven               |null        |[Marcel Dragomir]|
+-------------------------+------------+-----------------+
only showing top 4 rows



In [5]:
# 2. Selecting columns using col()
df.select(
    col("boardgame"),
    col("amazon_price"),
    col("credits.developer")
).show(n=4, truncate=False)

+-------------------------+------------+-----------------+
|boardgame                |amazon_price|developer        |
+-------------------------+------------+-----------------+
|Brass: Birmingham        |69.95       |null             |
|Pandemic Legacy: Season 1|76.97       |null             |
|Ark Nova                 |71.81       |null             |
|Gloomhaven               |null        |[Marcel Dragomir]|
+-------------------------+------------+-----------------+
only showing top 4 rows



In [28]:
# Renaming columns using aliasIO
df.select(
    col("boardgame").alias("Board Game"),
    col("amazon_price").alias("Amazon Price"),
    col("credits.developer").alias("Dev")
).show(n=4, truncate=False)

+-------------------------+------------+-----------------+
|Board Game               |Amazon Price|Dev              |
+-------------------------+------------+-----------------+
|Brass: Birmingham        |69.95       |null             |
|Pandemic Legacy: Season 1|76.97       |null             |
|Ark Nova                 |71.81       |null             |
|Gloomhaven               |null        |[Marcel Dragomir]|
+-------------------------+------------+-----------------+
only showing top 4 rows



In [29]:
# Using expressions in select

df.select(
    "boardgame",
    expr("amazon_price * 1.1").alias("Amazon Price with Tax"),
    expr("credits.developer").alias("Dev")
).show(n=4, truncate=False)

+-------------------------+---------------------+-----------------+
|boardgame                |Amazon Price with Tax|Dev              |
+-------------------------+---------------------+-----------------+
|Brass: Birmingham        |76.94500000000001    |null             |
|Pandemic Legacy: Season 1|84.667               |null             |
|Ark Nova                 |78.99100000000001    |null             |
|Gloomhaven               |null                 |[Marcel Dragomir]|
+-------------------------+---------------------+-----------------+
only showing top 4 rows



In [19]:
# Selecting with conditional Expressions

df.select(
    "boardgame",
    col("credits.developer").alias("Dev"),
    col("amazon_price"),
    expr("CASE WHEN amazon_price < 50 THEN 'Affordable' ELSE 'Expensive' END").alias("Price Category"),
    when(df['amazon_price'] < 80, 'Affordable').otherwise("Expensive").alias("Price Category"),
    when(col('amazon_price') < 80, 'Affordable').otherwise("Expensive").alias("Price Category2")
).show(n=4, truncate=False)

+-------------------------+-----------------+------------+--------------+--------------+---------------+
|boardgame                |Dev              |amazon_price|Price Category|Price Category|Price Category2|
+-------------------------+-----------------+------------+--------------+--------------+---------------+
|Brass: Birmingham        |null             |69.95       |Expensive     |Affordable    |Affordable     |
|Pandemic Legacy: Season 1|null             |76.97       |Expensive     |Affordable    |Affordable     |
|Ark Nova                 |null             |71.81       |Expensive     |Affordable    |Affordable     |
|Gloomhaven               |[Marcel Dragomir]|null        |Expensive     |Expensive     |Expensive      |
+-------------------------+-----------------+------------+--------------+--------------+---------------+
only showing top 4 rows



In [24]:
# Using selectExpr() for SQL-like Expressions

df.selectExpr(
    "boardgame",
    "amazon_price * 2 AS double_price"
).show(n=4, truncate=False)

+-------------------------+------------+
|boardgame                |double_price|
+-------------------------+------------+
|Brass: Birmingham        |139.9       |
|Pandemic Legacy: Season 1|153.94      |
|Ark Nova                 |143.62      |
|Gloomhaven               |null        |
+-------------------------+------------+
only showing top 4 rows



In [26]:
# Using select() with lit() to add constant columns

df.select(
    "boardgame",
    lit("Has Players").alias("Info"),
).show(n=4, truncate=False)

+-------------------------+-----------+
|boardgame                |Info       |
+-------------------------+-----------+
|Brass: Birmingham        |Has Players|
|Pandemic Legacy: Season 1|Has Players|
|Ark Nova                 |Has Players|
|Gloomhaven               |Has Players|
+-------------------------+-----------+
only showing top 4 rows



In [ ]:
# Selecting columns dynamically
# using *args

col_list = ["boardgame", "credits.developer", "credits.artists"]
df.select(
    *col_list
).show()

+--------------------+--------------------+--------------------+
|           boardgame|           developer|             artists|
+--------------------+--------------------+--------------------+
|   Brass: Birmingham|                null|[Gavan Brown, Lin...|
|Pandemic Legacy: ...|                null|   [Chris Quilliams]|
|            Ark Nova|                null|[Steffen Bieker, ...|
|          Gloomhaven|   [Marcel Dragomir]|[Alexandr Elichev...|
|Twilight Imperium...|[Dane Beltrami, J...|   [Scott Schomburg]|
|      Dune: Imperium|       [Paul Dennen]|[Clay Brooks, Bre...|
|   Terraforming Mars|                null|[Isaac Fryxelius,...|
|War of the Ring: ...|                null|[John Howe, Fabio...|
|Star Wars: Rebellion|    [Steven Kimball]|[Matt Allsopp, Da...|
|Dune: Imperium – ...|                null|[Clay Brooks, Der...|
|       Spirit Island|      [Ted Vessenes]|[Jason Behnke, Lo...|
|Gloomhaven: Jaws ...|[Zachary Cohn, Ma...|[Francesca Baeral...|
|        Gaia Project|   

In [32]:
# Selecting all columns except one

df.select([
        col for col in df.columns if col != "amazon_price"
    ]).show(4, truncate=False)

+-------------------------+---------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### Using `orderBy()` Function

In [36]:
col_list = ["boardgame", "credits.developer", "credits.artists"]
df2 = df.select(
    *col_list
)

df2.show(truncate=False, n=4)


+-------------------------+-----------------+---------------------------------------------------------------------------------------+
|boardgame                |developer        |artists                                                                                |
+-------------------------+-----------------+---------------------------------------------------------------------------------------+
|Brass: Birmingham        |null             |[Gavan Brown, Lina Cossette, David Forest, Gui Landgraf, Damien Mammoliti, Matt Tolman]|
|Pandemic Legacy: Season 1|null             |[Chris Quilliams]                                                                      |
|Ark Nova                 |null             |[Steffen Bieker, Loïc Billiau, Dennis Lohausen, Christof Tisch]                        |
|Gloomhaven               |[Marcel Dragomir]|[Alexandr Elichev, Josh T. McDowell, Alvaro Nebot]                                     |
+-------------------------+-----------------+-----------------

In [38]:
# Order by a single column (Ascending default)

df2.orderBy("boardgame").show(4, truncate=False)

+---------------------------------------+---------+----------------------------+
|boardgame                              |developer|artists                     |
+---------------------------------------+---------+----------------------------+
|null                                   |null     |null                        |
|13 Days: The Cuban Missile Crisis, 1962|null     |[Jacob Walker]              |
|1775: Rebellion                        |null     |[Jarek Nocoń, Steve Paschal]|
|1812: The Invasion of Canada           |null     |[Jarek Nocoń, Steve Paschal]|
+---------------------------------------+---------+----------------------------+
only showing top 4 rows



In [39]:
# Order by single column descending

df2.orderBy(col("boardgame").desc()).show(4, truncate=False)


+-------------------------------------+---------------------------+--------------------------------------+
|boardgame                            |developer                  |artists                               |
+-------------------------------------+---------------------------+--------------------------------------+
|스플렌더: Pokémon (Splendor: Pokémon)|null                       |null                                  |
|ito                                  |null                       |[326, Arclight, Nadia Carrim]         |
|burncycle                            |[Salem Scott, Josh Wielgus]|[Yoann Boissonnet, Anthony LeTourneau]|
|boop.                                |null                       |[Curt Covert]                         |
+-------------------------------------+---------------------------+--------------------------------------+
only showing top 4 rows



In [44]:
# Order by multiple columns

df2.orderBy(["boardgame", "developer"], ascending=[False, True]).show(4, truncate=False)

+-------------------------------------+---------------------------+--------------------------------------+
|boardgame                            |developer                  |artists                               |
+-------------------------------------+---------------------------+--------------------------------------+
|스플렌더: Pokémon (Splendor: Pokémon)|null                       |null                                  |
|ito                                  |null                       |[326, Arclight, Nadia Carrim]         |
|burncycle                            |[Salem Scott, Josh Wielgus]|[Yoann Boissonnet, Anthony LeTourneau]|
|boop.                                |null                       |[Curt Covert]                         |
+-------------------------------------+---------------------------+--------------------------------------+
only showing top 4 rows



In [42]:
# Order by multiple columns in custom order

df2.orderBy(
    col("boardgame").desc(),
    col("developer").asc()
).show(4, truncate=False)

+-------------------------------------+---------------------------+--------------------------------------+
|boardgame                            |developer                  |artists                               |
+-------------------------------------+---------------------------+--------------------------------------+
|스플렌더: Pokémon (Splendor: Pokémon)|null                       |null                                  |
|ito                                  |null                       |[326, Arclight, Nadia Carrim]         |
|burncycle                            |[Salem Scott, Josh Wielgus]|[Yoann Boissonnet, Anthony LeTourneau]|
|boop.                                |null                       |[Curt Covert]                         |
+-------------------------------------+---------------------------+--------------------------------------+
only showing top 4 rows



In [45]:
df2.sort(
    col("boardgame").desc(),
    col("developer").asc()
).show(4, truncate=False)

+-------------------------------------+---------------------------+--------------------------------------+
|boardgame                            |developer                  |artists                               |
+-------------------------------------+---------------------------+--------------------------------------+
|스플렌더: Pokémon (Splendor: Pokémon)|null                       |null                                  |
|ito                                  |null                       |[326, Arclight, Nadia Carrim]         |
|burncycle                            |[Salem Scott, Josh Wielgus]|[Yoann Boissonnet, Anthony LeTourneau]|
|boop.                                |null                       |[Curt Covert]                         |
+-------------------------------------+---------------------------+--------------------------------------+
only showing top 4 rows



#### How to use `withColumn()` Function

In [6]:
col_list = ["boardgame", "amazon_price", "credits.developer", "credits.artists"]
df2 = df.select(
    *col_list
)

df2.show(truncate=False, n=4)



+-------------------------+------------+-----------------+---------------------------------------------------------------------------------------+
|boardgame                |amazon_price|developer        |artists                                                                                |
+-------------------------+------------+-----------------+---------------------------------------------------------------------------------------+
|Brass: Birmingham        |69.95       |null             |[Gavan Brown, Lina Cossette, David Forest, Gui Landgraf, Damien Mammoliti, Matt Tolman]|
|Pandemic Legacy: Season 1|76.97       |null             |[Chris Quilliams]                                                                      |
|Ark Nova                 |71.81       |null             |[Steffen Bieker, Loïc Billiau, Dennis Lohausen, Christof Tisch]                        |
|Gloomhaven               |null        |[Marcel Dragomir]|[Alexandr Elichev, Josh T. McDowell, Alvaro Nebot]          

In [12]:
# Adding a new column using withColumn()

df_new = df2.withColumn("bonus", lit(1000)) # Adding a constant value column
df_new.show(4, truncate=False)

+-------------------------+------------+-----------------+---------------------------------------------------------------------------------------+-----+
|boardgame                |amazon_price|developer        |artists                                                                                |bonus|
+-------------------------+------------+-----------------+---------------------------------------------------------------------------------------+-----+
|Brass: Birmingham        |69.95       |null             |[Gavan Brown, Lina Cossette, David Forest, Gui Landgraf, Damien Mammoliti, Matt Tolman]|1000 |
|Pandemic Legacy: Season 1|76.97       |null             |[Chris Quilliams]                                                                      |1000 |
|Ark Nova                 |71.81       |null             |[Steffen Bieker, Loïc Billiau, Dennis Lohausen, Christof Tisch]                        |1000 |
|Gloomhaven               |null        |[Marcel Dragomir]|[Alexandr Elichev, Josh 

In [11]:
# Modifying an existing column

df_new = df2.withColumn('amazon_price', col('amazon_price') * 1.2)
df_new.show(4, truncate=False)

+-------------------------+-----------------+-----------------+---------------------------------------------------------------------------------------+
|boardgame                |amazon_price     |developer        |artists                                                                                |
+-------------------------+-----------------+-----------------+---------------------------------------------------------------------------------------+
|Brass: Birmingham        |83.94            |null             |[Gavan Brown, Lina Cossette, David Forest, Gui Landgraf, Damien Mammoliti, Matt Tolman]|
|Pandemic Legacy: Season 1|92.36399999999999|null             |[Chris Quilliams]                                                                      |
|Ark Nova                 |86.172           |null             |[Steffen Bieker, Loïc Billiau, Dennis Lohausen, Christof Tisch]                        |
|Gloomhaven               |null             |[Marcel Dragomir]|[Alexandr Elichev, Josh T

In [14]:
# Using withColumn with expression

df_expr = df2.withColumn("amazon_price_with_tax", expr("amazon_price + 10"))
df_expr.show(4, truncate=False)

+-------------------------+------------+-----------------+---------------------------------------------------------------------------------------+---------------------+
|boardgame                |amazon_price|developer        |artists                                                                                |amazon_price_with_tax|
+-------------------------+------------+-----------------+---------------------------------------------------------------------------------------+---------------------+
|Brass: Birmingham        |69.95       |null             |[Gavan Brown, Lina Cossette, David Forest, Gui Landgraf, Damien Mammoliti, Matt Tolman]|79.95                |
|Pandemic Legacy: Season 1|76.97       |null             |[Chris Quilliams]                                                                      |86.97                |
|Ark Nova                 |71.81       |null             |[Steffen Bieker, Loïc Billiau, Dennis Lohausen, Christof Tisch]                        |81.81    

In [18]:
# Changing the data type of a column
# Convert amazon_price to IntegerType

df_cast = df2.withColumn('amazon_price', col('amazon_price').cast('Integer'))
df_cast.show(4, truncate=False)

+-------------------------+------------+-----------------+---------------------------------------------------------------------------------------+
|boardgame                |amazon_price|developer        |artists                                                                                |
+-------------------------+------------+-----------------+---------------------------------------------------------------------------------------+
|Brass: Birmingham        |69          |null             |[Gavan Brown, Lina Cossette, David Forest, Gui Landgraf, Damien Mammoliti, Matt Tolman]|
|Pandemic Legacy: Season 1|76          |null             |[Chris Quilliams]                                                                      |
|Ark Nova                 |71          |null             |[Steffen Bieker, Loïc Billiau, Dennis Lohausen, Christof Tisch]                        |
|Gloomhaven               |null        |[Marcel Dragomir]|[Alexandr Elichev, Josh T. McDowell, Alvaro Nebot]          

In [20]:
# Renaming column by creating a new one and dropping the old one

df_renamed = df2.withColumn('amazon price', col('amazon_price')).drop('amazon_price')
df_renamed.show(n=4, truncate=False)

+-------------------------+-----------------+---------------------------------------------------------------------------------------+------------+
|boardgame                |developer        |artists                                                                                |amazon price|
+-------------------------+-----------------+---------------------------------------------------------------------------------------+------------+
|Brass: Birmingham        |null             |[Gavan Brown, Lina Cossette, David Forest, Gui Landgraf, Damien Mammoliti, Matt Tolman]|69.95       |
|Pandemic Legacy: Season 1|null             |[Chris Quilliams]                                                                      |76.97       |
|Ark Nova                 |null             |[Steffen Bieker, Loïc Billiau, Dennis Lohausen, Christof Tisch]                        |71.81       |
|Gloomhaven               |[Marcel Dragomir]|[Alexandr Elichev, Josh T. McDowell, Alvaro Nebot]                       

In [21]:
# using withColumnRenamed

df_renamed = df2.withColumnRenamed('amazon_price', 'amazon price')
df_renamed.show(n=4, truncate=False)

+-------------------------+------------+-----------------+---------------------------------------------------------------------------------------+
|boardgame                |amazon price|developer        |artists                                                                                |
+-------------------------+------------+-----------------+---------------------------------------------------------------------------------------+
|Brass: Birmingham        |69.95       |null             |[Gavan Brown, Lina Cossette, David Forest, Gui Landgraf, Damien Mammoliti, Matt Tolman]|
|Pandemic Legacy: Season 1|76.97       |null             |[Chris Quilliams]                                                                      |
|Ark Nova                 |71.81       |null             |[Steffen Bieker, Loïc Billiau, Dennis Lohausen, Christof Tisch]                        |
|Gloomhaven               |null        |[Marcel Dragomir]|[Alexandr Elichev, Josh T. McDowell, Alvaro Nebot]          

In [26]:
# Putting it all together

new_df = (
df2.withColumnRenamed('amazon_price', 'amazon price')
.withColumn('amazon price', col('amazon price').cast('integer'))
.withColumn('bonus', lit(1000))
.withColumn('check', when(col('amazon price') < 70, "Low").otherwise("High"))
)
new_df.show(n=4, truncate=False)

+-------------------------+------------+-----------------+---------------------------------------------------------------------------------------+-----+-----+
|boardgame                |amazon price|developer        |artists                                                                                |bonus|check|
+-------------------------+------------+-----------------+---------------------------------------------------------------------------------------+-----+-----+
|Brass: Birmingham        |69          |null             |[Gavan Brown, Lina Cossette, David Forest, Gui Landgraf, Damien Mammoliti, Matt Tolman]|1000 |Low  |
|Pandemic Legacy: Season 1|76          |null             |[Chris Quilliams]                                                                      |1000 |High |
|Ark Nova                 |71          |null             |[Steffen Bieker, Loïc Billiau, Dennis Lohausen, Christof Tisch]                        |1000 |High |
|Gloomhaven               |null        |[Marce